### Konstantina Andronikou

## This Notebook implements a comparison between Active and Passive

In [1]:
from allennlp_models.pretrained import load_predictor

In [2]:
import checklist
import logging
from checklist.editor import Editor
from checklist.perturb import Perturb
from checklist.test_types import MFT, INV, DIR
from checklist.expect import Expect
import csv

In [3]:
from checklist.pred_wrapper import PredictorWrapper

## Active

In [4]:
#removing warnings
logging.getLogger('allennlp.common.params').disabled = True 
logging.getLogger('allennlp.nn.initializers').disabled = True 
logging.getLogger('allennlp.modules.token_embedders.embedding').disabled = True 
logging.getLogger('urllib3.connectionpool').disabled = True 
logging.getLogger('allennlp.common.plugins').disabled = True 
logging.getLogger('allennlp.common.model_card').disabled = True 
logging.getLogger('allennlp.models.archival').disabled = True 
logging.getLogger('allennlp.data.vocabulary').disabled = True 
logging.getLogger('cached_path').disabled = True

srl_predictor = load_predictor('structured-prediction-srl')
output = srl_predictor.predict("Someone drove Mary to school")
output

{'verbs': [{'verb': 'drove',
   'description': '[ARG0: Someone] [V: drove] [ARG1: Mary] [ARGM-GOL: to school]',
   'tags': ['B-ARG0', 'B-V', 'B-ARG1', 'B-ARGM-GOL', 'I-ARGM-GOL']}],
 'words': ['Someone', 'drove', 'Mary', 'to', 'school']}

In [5]:
def predict_srl(data):
    
    pred = []
    for d in data:
        pred.append(srl_predictor.predict(d))
    return pred

predict_and_conf = PredictorWrapper.wrap_predict(predict_srl)

In [6]:
d = ["Someone drove Mary to school"]
pred = predict_and_conf(d)
pred

([{'verbs': [{'verb': 'drove',
     'description': '[ARG0: Someone] [V: drove] [ARG1: Mary] [ARGM-GOL: to school]',
     'tags': ['B-ARG0', 'B-V', 'B-ARG1', 'B-ARGM-GOL', 'I-ARGM-GOL']}],
   'words': ['Someone', 'drove', 'Mary', 'to', 'school']}],
 array([1.]))

In [7]:
def format_srl(x, pred, conf, label=None, meta=None):
    
    return pred['verbs'][0]['description']

In [8]:
def get_arg(pred, arg_target='ARG0'):
    predicate_arguments = pred['verbs'][0]
    #print(predicate_arguments)
    words = pred['words']
    tags = predicate_arguments['tags']
    
    arg_list = []
    for t, w in zip(tags, words):
        arg = t
        if '-' in t:
            arg = t.split('-')[1]
        if arg == arg_target:
            arg_list.append(w)
    arg_set = set(arg_list)
    return arg_set

In [9]:
def format_srl(x, pred, conf, label=None, meta=None):
    results = []
    predicate_structure = pred['verbs'][0]['description']
        
    return predicate_structure

In [10]:
def found_arg0_people(x, pred, conf, label=None, meta=None):
    
    # people should be recognized as arg1

    people = set([meta['first_name']])
    arg_0 = get_arg(pred, arg_target='ARG0')

    if arg_0 == people:
        pass_ = True
    else:
        pass_ = False
    return pass_


expect_arg0 = Expect.single(found_arg0_people)

In [11]:
editor = Editor()

# create examples
t = editor.template('{first_name} drove John to school.', meta=True, nsamples=1000, remove_duplicates = True)

#print(type(t))

for k, v in t.items():
    print(k, v)

meta [{'first_name': 'Francis'}, {'first_name': 'Louis'}, {'first_name': 'Hugh'}, {'first_name': 'Louis'}, {'first_name': 'Dan'}, {'first_name': 'Julie'}, {'first_name': 'Carolyn'}, {'first_name': 'Maria'}, {'first_name': 'Ray'}, {'first_name': 'Helen'}, {'first_name': 'Alexandra'}, {'first_name': 'Matthew'}, {'first_name': 'Dorothy'}, {'first_name': 'Paul'}, {'first_name': 'Alexander'}, {'first_name': 'Paul'}, {'first_name': 'Sally'}, {'first_name': 'Jonathan'}, {'first_name': 'Cynthia'}, {'first_name': 'Lauren'}, {'first_name': 'Wendy'}, {'first_name': 'Jack'}, {'first_name': 'Pamela'}, {'first_name': 'Scott'}, {'first_name': 'Donald'}, {'first_name': 'Deborah'}, {'first_name': 'Julie'}, {'first_name': 'Annie'}, {'first_name': 'Kevin'}, {'first_name': 'Harold'}, {'first_name': 'John'}, {'first_name': 'Nancy'}, {'first_name': 'Karen'}, {'first_name': 'Roy'}, {'first_name': 'Angela'}, {'first_name': 'Larry'}, {'first_name': 'Marilyn'}, {'first_name': 'Johnny'}, {'first_name': 'Virginia

In [12]:
with open('Dataset_BiLSTM/Active.txt', 'w') as f:
    print(t.data, file = f)

In [13]:
# initialize a rest object
test = MFT(**t, name = 'detect_arg0_name_default_position', expect=expect_arg0)
output = test.run(predict_and_conf)
test.summary(format_example_fn=format_srl)
i = test.results['preds']
expect_for_i = test.results['expect_results']
with open ('Output_BiLSTM/Active.csv','w') as f:
    writer = csv.writer(f)
    for result, exp in zip(i, expect_for_i):
        
        case = result['words']
        exp = exp
        writer.writerow([case, exp])
        print(case,exp)

Predicting 1000 examples
Test cases:      1000
Fails (rate):    5 (0.5%)

Example fails:
[ARGM-MNR: Kathy] [V: drove] [ARG1: John] [ARG2: to school] .
----
[ARGM-MNR: Kathy] [V: drove] [ARG1: John] [ARG2: to school] .
----
[ARGM-MNR: Kathy] [V: drove] [ARG1: John] [ARG2: to school] .
----
['Francis', 'drove', 'John', 'to', 'school', '.'] [ True]
['Louis', 'drove', 'John', 'to', 'school', '.'] [ True]
['Hugh', 'drove', 'John', 'to', 'school', '.'] [ True]
['Louis', 'drove', 'John', 'to', 'school', '.'] [ True]
['Dan', 'drove', 'John', 'to', 'school', '.'] [ True]
['Julie', 'drove', 'John', 'to', 'school', '.'] [ True]
['Carolyn', 'drove', 'John', 'to', 'school', '.'] [ True]
['Maria', 'drove', 'John', 'to', 'school', '.'] [ True]
['Ray', 'drove', 'John', 'to', 'school', '.'] [ True]
['Helen', 'drove', 'John', 'to', 'school', '.'] [ True]
['Alexandra', 'drove', 'John', 'to', 'school', '.'] [ True]
['Matthew', 'drove', 'John', 'to', 'school', '.'] [ True]
['Dorothy', 'drove', 'John', 'to'

In [14]:
# the following piece of code was adapted from https://howtodoinjava.com/examples/python-print-to-file/
import sys
 
original_stdout = sys.stdout  
 
with open('Predictions_BiLSTM/false_sentences_Active.txt', 'a') as f:
    sys.stdout = f 
    print(test.summary(format_example_fn=format_srl), file = f)
    # Reset the standard output
    sys.stdout = original_stdout 

## PASSIVE

In [15]:
def format_srl(x, pred, conf, label=None, meta=None):
    
    return pred['verbs'][1]['description'] #second verb in this case 

In [16]:
def get_arg(pred, arg_target='ARG1'):
    predicate_arguments = pred['verbs'][1]
    #print(predicate_arguments)
    words = pred['words']
    tags = predicate_arguments['tags']
    
    arg_list = []
    for t, w in zip(tags, words):
        arg = t
        if '-' in t:
            arg = t.split('-')[1]
        if arg == arg_target:
            arg_list.append(w)
    arg_set = set(arg_list)
    return arg_set

In [17]:
def format_srl(x, pred, conf, label=None, meta=None):
    results = []
    predicate_structure = pred['verbs'][1]['description'] #second verb 
        
    return predicate_structure

In [18]:
def found_arg1_people(x, pred, conf, label=None, meta=None):
    
    # people should be recognized as arg1

    people = set([meta['first_name']])
    arg_1 = get_arg(pred, arg_target='ARG1')

    if arg_1 == people:
        pass_ = True
    else:
        pass_ = False
    return pass_


expect_arg1 = Expect.single(found_arg1_people)

In [19]:
editor = Editor()

# create examples
t = editor.template('{first_name} got driven to school by John.', meta=True, nsamples=1000, remove_duplicates = True)

#print(type(t))

for k, v in t.items():
    print(k, v)

meta [{'first_name': 'Janet'}, {'first_name': 'Evelyn'}, {'first_name': 'Larry'}, {'first_name': 'Charlotte'}, {'first_name': 'Edwin'}, {'first_name': 'Lauren'}, {'first_name': 'Jason'}, {'first_name': 'Gary'}, {'first_name': 'Christine'}, {'first_name': 'Margaret'}, {'first_name': 'Don'}, {'first_name': 'Louis'}, {'first_name': 'Jill'}, {'first_name': 'Edith'}, {'first_name': 'Peter'}, {'first_name': 'Kathy'}, {'first_name': 'Bruce'}, {'first_name': 'Christopher'}, {'first_name': 'Eleanor'}, {'first_name': 'Peter'}, {'first_name': 'Matt'}, {'first_name': 'Kathy'}, {'first_name': 'Frederick'}, {'first_name': 'Margaret'}, {'first_name': 'Paul'}, {'first_name': 'Alexandra'}, {'first_name': 'Ellen'}, {'first_name': 'Alice'}, {'first_name': 'Elizabeth'}, {'first_name': 'Louise'}, {'first_name': 'Rebecca'}, {'first_name': 'Diane'}, {'first_name': 'Melissa'}, {'first_name': 'Jean'}, {'first_name': 'Fiona'}, {'first_name': 'Marilyn'}, {'first_name': 'Maria'}, {'first_name': 'Donald'}, {'first

In [20]:
with open('Dataset_BiLSTM/Passive.txt', 'w') as f:
    print(t.data, file = f)

In [21]:
# initialize a rest object
test = MFT(**t, name = 'detect_arg1_name_default_position', expect=expect_arg1)
output = test.run(predict_and_conf)
test.summary(format_example_fn=format_srl)
i = test.results['preds']
expect_for_i = test.results['expect_results']
with open ('Output_BiLSTM/Passive.csv','w') as f:
    writer = csv.writer(f)
    for result, exp in zip(i, expect_for_i):
        
        case = result['words']
        exp = exp
        writer.writerow([case, exp])
        print(case,exp)

Predicting 1000 examples
Test cases:      1000
Fails (rate):    112 (11.2%)

Example fails:
Kathy got [V: driven] [ARG2: to school] [ARG0: by John] .
----
Kathy got [V: driven] [ARG2: to school] [ARG0: by John] .
----
Ken got [V: driven] [ARG2: to school] [ARG0: by John] .
----
['Janet', 'got', 'driven', 'to', 'school', 'by', 'John', '.'] [ True]
['Evelyn', 'got', 'driven', 'to', 'school', 'by', 'John', '.'] [ True]
['Larry', 'got', 'driven', 'to', 'school', 'by', 'John', '.'] [ True]
['Charlotte', 'got', 'driven', 'to', 'school', 'by', 'John', '.'] [False]
['Edwin', 'got', 'driven', 'to', 'school', 'by', 'John', '.'] [False]
['Lauren', 'got', 'driven', 'to', 'school', 'by', 'John', '.'] [ True]
['Jason', 'got', 'driven', 'to', 'school', 'by', 'John', '.'] [ True]
['Gary', 'got', 'driven', 'to', 'school', 'by', 'John', '.'] [ True]
['Christine', 'got', 'driven', 'to', 'school', 'by', 'John', '.'] [False]
['Margaret', 'got', 'driven', 'to', 'school', 'by', 'John', '.'] [ True]
['Don', '

In [22]:
# the following piece of code was adapted from https://howtodoinjava.com/examples/python-print-to-file/
import sys
 
original_stdout = sys.stdout  
 
with open('Predictions_BiLSTM/false_sentences_Passive.txt', 'a') as f:
    sys.stdout = f 
    print(test.summary(format_example_fn=format_srl), file = f)
    # Reset the standard output
    sys.stdout = original_stdout 